<a href="https://colab.research.google.com/github/Kakaoko/Sandwich-Panel-Investigation/blob/master/Sandwich_Panel_Transfer_Matrix_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import matplotlib.pyplot as plt

#Material property
#PLA for 3D Printing
global ela_pla, den_pla, c_pla, c_air, zc_air, theta
e_pla = 3.5 * 10**9
d_pla = 1240
c_pla = 2220
c_air = 344
zc_air = 409.4

#Thin Elastic Panel Transfer Matrix
class panel :
    def __init__(self, f, b, h, L) :
        self.f = f
        self.b = b
        self.h = h
        self.L = L

    def w(self) :
        return 2 * np.pi * self.f

    def k_pla(self) :
        return 2 * np.pi * self.f / c_pla

    def k_air(self) :
        return 2 * np.pi * self.f / c_air

    def B(self) :
        return e_pla * self.b * self.h**3 / (12 * self.L)

    def Zp(self) :
        return panel(self.f, self.b, self.h, self.L).B() * (panel(self.f, self.b, self.h, self.L).k_air()**4 * np.sin(theta)**4 - panel(self.f, self.b, self.h, self.L).k_pla()**4) / (panel(self.f, self.b, self.h, self.L).w() * 1j)

    def T_panel(self) :
        return np.array([[1, panel(self.f, self.b, self.h, self.L).Zp()], [0 ,1]])

#Fluid Layer Transfer Matrix
class fluid :
    def __init__(self, f, d) :
        self.f = f
        self.d = d

    def w(self) :
        return 2 * np.pi * self.f

    def k_air(self) :
        return 2 * np.pi * self.f / c_air

    def m00(self) :
        return np.cosh(fluid(self.f, self.d).k_air() * self.d * np.cos(theta) * 1j)

    def m01(self) :
        return zc_air * np.sinh(fluid(self.f, self.d).k_air() * self.d * np.cos(theta) * 1j) / np.cos(theta)

    def m10(self) :
        return (np.sin(fluid(self.f, self.d).k_air() * self.d * np.cos(theta) * 1j)) * np.cos(theta) / zc_air

    def m11(self) :
        return fluid(self.f, self.d).m00()

    def T_fluid(self) :
        return np.array([[fluid(self.f, self.d).m00(), fluid(self.f, self.d).m01()], [fluid(self.f, self.d).m10(), fluid(self.f, self.d).m11()]])

#Porous Layer Transfer Matrix
class porous :
    def __init__(self, f, d, b, h, sigma) :
        self.f = f
        self.d = d
        self.b = b
        self.h = h
        self.sigma = sigma

    def w(self) :
        return 2 * np.pi * self.f

    def k_air(self) :
        return 2 * np.pi * self.f / c_air

    def mass(self) :
        return b * h * d * den_pla * (1 - sigma)

    def m00(self) :
        return np.cosh(porous(self.f, self.d, self.b, self.h, self.sigma).k_air() * self.d * np.cos(theta) * 1j)

    def m01(self) :
        return zc_air * np.sinh(porous(self.f, self.d, self.b, self.h, self.sigma).k_air() * self.d * np.cos(theta) * 1j) / (np.cos(theta) * self.sigma)

    def m10(self) :
        return (np.sin(porous(self.f, self.d, self.b, self.h, self.sigma).k_air() * self.d * np.cos(theta) * 1j)) * np.cos(theta) * self.sigma / zc_air

    def m11(self) :
        return porous(self.f, self.d, self.b, self.h, self.sigma).m00()

    def T_porous(self) :
        return np.array([[porous(self.f, self.d, self.b, self.h, self.sigma).m00(), porous(self.f, self.d, self.b, self.h, self.sigma).m01()], [porous(self.f, self.d, self.b, self.h, self.sigma).m10(), porous(self.f, self.d, self.b, self.h, self.sigma).m11()]])

    def T_mass(self) :
        return np.array([[1, porous(self.f, self.d, self.b, self.h, self.sigma).w() * porous(self.f, self.d, self.b, self.h, self.sigma).mass() * 1j], [0, 1]])

#Shock Absorber Transfer Matrix
class shock_ab :
    def __init__(self, f, c, k) :
        self.f = f
        self.c = c
        self.k = k

    def w(self) :
        return 2 * np.pi * self.f

    def m10(self) :
        return (shock_ab(self.f, self.c, self.k).w() * 1j) / (self.k + shock_ab(self.f, self.c, self.k).w() * self.c * 1j)

    def T_shock_ab(self) :
        return np.array([[1, 0], [shock_ab(self.f, self.c, self.k).m10(), 1]])

#Graph Plotting Fuction
def graph_plot(TL_1, TL_2) :
    plt.plot(np.arange(50, 1600), TL_1, ":", label = "Typical Sandwich Panel")
    plt.plot(np.arange(50, 1600), TL_2, "--", label = "Sandwich Panel with Shock Absorber")
    plt.title("Sandwich Panel Transmission Loss(dB)")
    plt.xlabel("Frequency(Hz)")
    plt.ylabel("TL(dB)")
    plt.legend(loc = "upper right")
    plt.savefig("Sandwich Panel Transmission Loss(dB)", dpi = 300)
    return plt.show()

#Parameter Input
def para_input() :
    print("< Panel -> Air -> Porous -> Air -> Panel >\n")
    b = float(input("width of cross section(m) : "))
    h = float(input("height of cross section(m) : "))
    L = float(input("thickness of panel(m) : "))
    d = float(input("thickness of air gap(m) : "))
    sigma = float(input("porosity(0 ~ 1) : "))
    theta = float(input("angle of incident(0 ~ 1.57rad) : "))
    print()
    return b, h, L, d, sigma, theta

b, h, L, d, sigma, theta = para_input()

#Panel -> Air -> Porous -> Air -> Panel
tau_1 = []
for f in range(50, 1600) :
    T = np.matmul(np.matmul(np.matmul(panel(f, b, h, L).T_panel(), fluid(f, d).T_fluid()), np.matmul(porous(f, d, b, h, sigma).T_porous(), fluid(f, d).T_fluid())), panel(f, b, h, L).T_panel())
    tau_1.append(4 * np.absolute(T[0][0] + T[0][1] / zc_air + zc_air * T[1][0] + T[1][1])**-2)

TL_1 = []
for i in range(len(tau_1)) :
    TL_1.append(- 10 * np.log10(tau_1[i]))

#Panel -> Air -> Porous -> Shock Absorber -> Porous -> Panel
#Parameter Input (Shock Absorber)
def para_input_shock_absorber() :
    print("< Panel -> Air -> Porous -> Shock Absorber -> Porous -> Panel >\n")
    c = float(input("Damping Coefficient : "))
    k = float(input("Spring Coefficient) : "))
    return c, k

c, k = para_input_shock_absorber()

Area, Area_inv = np.array([[b * h, 0], [0, 1]]), np.array([[1 / (b * h), 0], [0, 1]])

tau_2 = []
for f in range(50, 1600) :
    T = np.matmul(np.matmul(np.matmul(np.matmul(panel(f, b, h, L).T_panel(), fluid(f, d).T_fluid()), np.matmul(porous(f, d, b, h, sigma).T_porous(), Area)), np.matmul(shock_ab(f, c, k).T_shock_ab(), Area_inv)), np.matmul(porous(f, d, b, h, sigma).T_porous(), panel(f, b, h, L).T_panel()))
    tau_2.append(4 * np.absolute(T[0][0] + T[0][1] / zc_air + zc_air * T[1][0] + T[1][1])**-2)

TL_2 = []
for i in range(len(tau_2)) :
    TL_2.append(- 10 * np.log10(tau_2[i]))

#Graph Plotting
graph_plot(TL_1, TL_2)